In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2
import tensorflow

from tensorflow.keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import os
import seaborn as sns
from keras.applications.vgg16 import VGG16

In [ ]:
#pip install opencv-python

In [ ]:
#pip install tensorflow

In [ ]:
#pip install google-colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read input images and assign labels based on folder names
#print(os.listdir("C:\Users\user\Malimg\train"))

SIZE = 128  #Resize images

#Capture training data and labels into respective lists
train_images = []
train_labels = [] 

for directory_path in glob.glob("/content/drive/MyDrive/Dataset/malimg_paper_dataset_imgs/train/*"):
    label = directory_path.split("/")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)

#Convert lists to arrays        
train_images = np.array(train_images)
train_labels = np.array(train_labels)

Streaming output truncated to the last 5000 lines.
/content/drive/MyDrive/Dataset/malimg_paper_dataset_imgs/train/Allaple.A/051ab3a970ab2a9d150df473caad7f6b.png
/content/drive/MyDrive/Dataset/malimg_paper_dataset_imgs/train/Allaple.A/05e94eef82b828f187bc3fe06434e2a5.png
/content/drive/MyDrive/Dataset/malimg_paper_dataset_imgs/train/Allaple.A/062ee42d7e9e3dae54c92b6e7b1773a3.png
/content/drive/MyDrive/Dataset/malimg_paper_dataset_imgs/train/Allaple.A/001ac0895271fe587d404b41d1a369f5.png
/content/drive/MyDrive/Dataset/malimg_paper_dataset_imgs/train/Allaple.A/03ab1430f5e3fefe751f27f4c3ce0153.png
/content/drive/MyDrive/Dataset/malimg_paper_dataset_imgs/train/Allaple.A/050558ba47a924827c62218bda1edba4.png
/content/drive/MyDrive/Dataset/malimg_paper_dataset_imgs/train/Allaple.A/01d694d598e4c408a209e9ab740bcc9b.png
/content/drive/MyDrive/Dataset/malimg_paper_dataset_imgs/train/Allaple.A/001136f832a1eea98ab2a0d141d8a8ea.png
/content/drive/MyDrive/Dataset/malimg_paper_dataset_imgs/train/Allapl

In [ ]:
# Capture test/validation data and labels into respective lists

test_images = []
test_labels = [] 
for directory_path in glob.glob("/content/drive/MyDrive/Dataset/malimg_paper_dataset_imgs/val/*"):
    fruit_label = directory_path.split("/")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(fruit_label)

#Convert lists to arrays                
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)
#print(train_labels_encoded)

In [ ]:
#Split data into test and train datasets (already split but assigning to meaningful convention)
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded
#print(y_train)


In [ ]:
# Normalize pixel values to between 0 and 1

x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
#One hot encode y values for neural network. 
from tensorflow.keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)


In [ ]:
# Load model wothout classifier/fully connected layers
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
# Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in VGG_model.layers:
    layer.trainable = False
VGG_model.summary()  # Trainable parameters will be 0

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [ ]:
# Now, let us use features from convolutional network for XBG and SVM

feature_extractor=VGG_model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

In [ ]:
# Send test data through same feature extractor process
X_test_feature = VGG_model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

In [ ]:
DNN_model= Sequential()

In [ ]:
DNN_model.add(VGG_model)
DNN_model.add(Flatten())
DNN_model.add(Dense(128, activation = 'relu'))
DNN_model.add(Dense(25, activation = 'softmax'))

In [ ]:
DNN_model.compile(loss ='categorical_crossentropy', optimizer='adam',metrics =['acc'])
DNN_model.fit(x_train, y_train_one_hot, epochs=50)

Epoch 1/50
234/234 [==============================] - 25s 101ms/step - loss: 0.3373 - acc: 0.9188
Epoch 2/50
234/234 [==============================] - 24s 101ms/step - loss: 0.1246 - acc: 0.9635
Epoch 3/50
234/234 [==============================] - 23s 100ms/step - loss: 0.1045 - acc: 0.9676
Epoch 4/50
234/234 [==============================] - 24s 101ms/step - loss: 0.0800 - acc: 0.9748
Epoch 5/50
234/234 [==============================] - 24s 101ms/step - loss: 0.0779 - acc: 0.9727
Epoch 6/50
234/234 [==============================] - 24s 102ms/step - loss: 0.0708 - acc: 0.9752
Epoch 7/50
234/234 [==============================] - 24s 102ms/step - loss: 0.0606 - acc: 0.9772
Epoch 8/50
234/234 [==============================] - 24s 102ms/step - loss: 0.0623 - acc: 0.9780
Epoch 9/50
234/234 [==============================] - 24s 102ms/step - loss: 0.0519 - acc: 0.9816
Epoch 10/50
234/234 [==============================] - 24s 101ms/step - loss: 0.0516 - acc: 0.9807
Epoch 11/50
234/234

In [ ]:
# Print overall accuracy
from sklearn import metrics

prediction = DNN_model.evaluate(x_test, y_test_one_hot , verbose=0)
print("DNN %s: %.2f%%" % (DNN_model.metrics_names[1], prediction[1]*100))


DNN acc: 98.51%
